# V827 spot-checking

In [ ]:
#! cat /Users/obsidian/Desktop/defaults.py

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../sf/m115/output/subsub_m115/run01/spec_config.csv')
#df = pd.read_csv('../sf/V827Tau/m102/output/mix_emcee/run01/models_ff-05_50_95.csv')

In [ ]:
df.head(2)

In [ ]:
plt.figure(figsize=(10,6))
plt.step(df.wl, df.data, label='data')
scale = 1#df.data.median()/df.model_composite.median()
plt.plot(df.wl, df.model_composite*scale, label='Composite')
#plt.plot(df.wl, noised, label='Noised up')
#plt.plot(df.wl, df.model_cool50, label='Starspot')
#plt.plot(df.wl, df.model_hot50, label='Ambient')
plt.legend(loc='best')

In [ ]:
np.log10(scale)

In [ ]:
CC = np.load('../sf/m115/output/subsub_m115/run01/CC.npy')

noised = np.random.multivariate_normal(df.model_composite, CC)

R_GP = df.model_composite-noised
R_dat = df.model_composite-df.data

plt.plot(df.wl, R_dat)
plt.plot(df.wl, R_GP)

In [ ]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size//2:]

In [ ]:
plt.plot(autocorr(R_dat.values), label='Data - model mismatches')
plt.plot(autocorr(R_GP.values), label = 'GP')
plt.legend(loc='best')

In [ ]:
plt.figure(figsize=(10,6))
plt.step(df.wl, df.data, label='data')
scale = 1.0 #df.data.median()/df.model_composite.median()
plt.plot(df.wl, df.model_composite*scale, label='Composite')
plt.plot(df.wl, noised, label='Noised up')
plt.plot(df.wl, df.model_cool50, label='Starspot')
plt.plot(df.wl, df.model_hot50, label='Ambient')
plt.legend(loc='best')

Need to adjust `logOmega`:

In [ ]:
logOmega = -5.42
logOmega2 = -5.20
f = 10**logOmega2 / (10**logOmega2 + 10**logOmega)
f

What about the MCMC chains?

In [ ]:
ws = np.load("../sf/m115/output/subsub_m115/run01/temp_emcee_chain.npy")

In [ ]:
max_obs = ws.any(axis=(0,2)).sum()

In [ ]:
max_obs

In [ ]:
burned = ws[:, 0:max_obs,:]
xs, ys, zs = burned.shape
fc = burned.reshape(xs*ys, zs)
nx, ny = fc.shape

In [ ]:
from matplotlib.ticker import MaxNLocator

In [ ]:
label = [r"$T_{\mathrm{eff}}$", r"$\log{g}$", '[Fe/H]',r"$v_z$", r"$v\sin{i}$", r"$\log{\Omega}$", 
         r"$T_{\mathrm{eff}, 2}$", r"$\log{\Omega_2}$",
         r"$c^1$", r"$c^2$", r"$c^3$", r"sigAmp", r"logAmp", r"$l$"] 

In [ ]:
fig, axes = plt.subplots(14, 1, sharex=True, figsize=(8, 14))
for i in range(0, 14, 1):
    axes[i].plot(burned[:, :, i].T, color="k", alpha=0.2)
    axes[i].yaxis.set_major_locator(MaxNLocator(5))
    axes[i].set_ylabel(label[i])

axes[13].set_xlabel("step number")
fig.tight_layout(h_pad=0.0)

What do those final 200 samples look like in spectra?

In [ ]:
df = pd.read_csv('../sf/m115/output/subsub_m115/run01/models_ff-05_50_95.csv')

In [ ]:
df.head()

In [ ]:
CC = np.load('../sf/m115/output/subsub_m115/run01/CC.npy')

noised = np.random.multivariate_normal(df.model_comp50, CC)

R_GP = df.model_comp50-noised
R_dat = df.model_comp50-df.data

plt.plot(df.wl, R_dat)
plt.plot(df.wl, R_GP)

In [ ]:
plt.figure(figsize=(10,6))
plt.step(df.wl, df.data, label='data')
plt.plot(df.wl, df.model_comp95, label='Composite')
plt.plot(df.wl, df.model_comp05, label='Composite')
#plt.plot(df.wl, df.model_comp05, label='Composite $-2\sigma$')
#plt.plot(df.wl, df.model_comp95, label='Composite $+2\sigma$')
#plt.plot(df.wl, noised, label='Noised up')
plt.plot(df.wl, df.model_cool95, label='Starspot')
plt.plot(df.wl, df.model_cool05, label='Starspot')
#plt.plot(df.wl, df.model_hot50, label='Ambient')
plt.legend(loc='best')

The end.